In [ ]:
! pip install petpy
import petpy
import os
import pandas as pd
import urllib.request
import urllib.error
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from pandas import json_normalize

In [ ]:
pf = petpy.Petfinder(key='ZR8m4gfixSRxbRgI5vHhVIP5hNnKjQD97VCUXkjSqFwlMCAe3Z',secret='OO4M11Yvrxb89Bljhf4hBO5CXyqsuO7YPF1A6Zna')

In [ ]:
dogs = pf.animal_types('dog')
dog_breeds = pf.breeds('dog')
dog_breeds['breeds']['dog']

['Affenpinscher',
 'Afghan Hound',
 'Airedale Terrier',
 'Akbash',
 'Akita',
 'Alaskan Malamute',
 'American Bulldog',
 'American Bully',
 'American Eskimo Dog',
 'American Foxhound',
 'American Hairless Terrier',
 'American Staffordshire Terrier',
 'American Water Spaniel',
 'Anatolian Shepherd',
 'Appenzell Mountain Dog',
 'Aussiedoodle',
 'Australian Cattle Dog / Blue Heeler',
 'Australian Kelpie',
 'Australian Shepherd',
 'Australian Terrier',
 'Basenji',
 'Basset Hound',
 'Beagle',
 'Bearded Collie',
 'Beauceron',
 'Bedlington Terrier',
 'Belgian Shepherd / Laekenois',
 'Belgian Shepherd / Malinois',
 'Belgian Shepherd / Sheepdog',
 'Belgian Shepherd / Tervuren',
 'Bernedoodle',
 'Bernese Mountain Dog',
 'Bichon Frise',
 'Black and Tan Coonhound',
 'Black Labrador Retriever',
 'Black Mouth Cur',
 'Black Russian Terrier',
 'Bloodhound',
 'Blue Lacy',
 'Bluetick Coonhound',
 'Boerboel',
 'Bolognese',
 'Border Collie',
 'Border Terrier',
 'Borzoi',
 'Boston Terrier',
 'Bouvier des Fl

In [ ]:
germanshepard.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'organization_animal_id',
       'photos', 'videos', 'status', 'status_changed_at', 'published_at',
       'distance', 'breeds.primary', 'breeds.secondary', 'breeds.mixed',
       'breeds.unknown', 'colors.primary', 'colors.secondary',
       'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'primary_photo_cropped.small', 'primary_photo_cropped.medium',
       'primary_photo_cropped.large', 'primary_photo_cropped.full',
       'contact.email', 'contact.phone', 'contact.address.address1',
       'contact.address.address2', 'contact.address.city',
       'contact.address.state', 'contact.address.postcode',
       'contact.address.country', 'animal_id', 'animal_type',

In [ ]:
def check_df_compat(results):
    key = list(results.keys())[0]
    results_df = json_normalize(results[key])
    problemcols = ['_links.organization.href','_links.self.href','_links.type.href']
    i = 0
    
    if key == 'animals':
      
      if '_links.organization.href' in results_df.columns:
        results_df['_links.organization.href'] = results_df['_links.organization.href']\
            .str.replace('/v2/organizations/', '')
        i +=1
      if '_links.self.href' in results_df.columns:
        results_df['_links.self.href'] = results_df['_links.self.href'].str.replace('/v2/animals/', '')
        i +=1
      if '_links.type.href' in results_df.columns:
        results_df['_links.type.href'] = results_df['_links.type.href'].str.replace('/v2/types/', '')
        i +=1
      if i == 3:
        results_df.rename(columns={'_links.organization.href': 'organization_id',
                                    '_links.self.href': 'animal_id',
                                    '_links.type.href': 'animal_type'}, inplace=True)

    if key == 'organizations':
        del results_df['_links.animals.href']
        results_df['_links.self.href'] = results_df['_links.self.href'].str.replace('/v2/organizations/', '')

        results_df.rename(columns={'_links.self.href': 'organization_id'}, inplace=True)

    return results_df


def get_dogs_breed(dogbreed):
    zipcodes = ['42.108133, -75.227792','32.038031, -84.544198', '42.238408, -93.860605']
    '''
    42.108133, -75.227792
    32.038031, -84.544198
    42.238408, -93.860605
    30.384399, -101.594980
    38.901429, -112.493417
    51.462330, -106.868417
    56.044095, -72.096615
    '''
    i = 0
    listofdf = []
    newdogs = []

    while i < len(zipcodes):
      print(i)
      if len(newdogs) < 1500:
        newdogsraw = pf.animals(location=zipcodes[i], animal_type='dog',breed = dogbreed, distance=500, results_per_page=100, pages=5)
        if len(newdogsraw['animals']) == 0:
          i += 1
          continue

        newdogs = check_df_compat(newdogsraw)
        newdogs = newdogs[newdogs['photos'].str.len() != 0]
        newdogs = newdogs.loc[:,~newdogs.columns.duplicated()]
        listofdf.append(newdogs)
        
        if len(listofdf) > 1:
          newdogs = pd.concat(listofdf,ignore_index=True, sort=False)
        print(len(newdogs))
      i += 1
    print(dogbreed)
    return newdogs

In [ ]:
def get_dogs_location(locationlat):
    newdogsraw = pf.animals(location=locationlat, animal_type='dog', distance=500, results_per_page=100, pages=221)
    
    if len(newdogsraw['animals']) == 0:
      return None

    newdogs = check_df_compat(newdogsraw)
    newdogs = newdogs[newdogs['photos'].str.len() != 0]
    newdogs = newdogs.loc[:,~newdogs.columns.duplicated()]

    print(locationlat)
    return newdogs

In [ ]:
latlonlist = ['42.108133, -75.227792','32.038031, -84.544198', '42.238408, -93.860605','30.384399, -101.594980', '38.901429, -112.493417','51.462330, -106.868417','56.044095, -72.096615']

dog_df_list = [get_dogs_location(latlon) for latlon in latlonlist]

dog_df = pd.concat(dog_df_list,ignore_index=True, sort=False)


pages parameter exceeded maximum number of available pages available from the Petfinder API. As a result, the maximum number of pages 195 was returned
42.108133, -75.227792
32.038031, -84.544198
pages parameter exceeded maximum number of available pages available from the Petfinder API. As a result, the maximum number of pages 151 was returned
42.238408, -93.860605
pages parameter exceeded maximum number of available pages available from the Petfinder API. As a result, the maximum number of pages 146 was returned
30.384399, -101.594980
pages parameter exceeded maximum number of available pages available from the Petfinder API. As a result, the maximum number of pages 154 was returned
38.901429, -112.493417
pages parameter exceeded maximum number of available pages available from the Petfinder API. As a result, the maximum number of pages 5 was returned
51.462330, -106.868417
pages parameter exceeded maximum number of available pages available from the Petfinder API. As a result, the ma

In [ ]:
def get_images(df):

    # get rid of duplicates
    df_nodup = df.loc[df.astype(str).drop_duplicates().index]

    # Keep only the columns that contain 'age' ,'id', 'breed', and 'photo'
    photos = df[df.columns[df.columns.str.contains('name|size|gender|postcode|description|age|id|breed|photo')]]
    photos = photos.drop(['photos','organization_id','organization_animal_id','videos', 'primary_photo_cropped','primary_photo_cropped.small','primary_photo_cropped.medium',"primary_photo_cropped.large"], axis =1)

    # sort by age
    photos_sorted = photos.sort_values(by=['age'])

    #reindex
    photos_sorted = photos_sorted.reset_index()
    del photos_sorted['index']
    
    return photos_sorted

In [ ]:
finaldf = get_images(dog_df)

In [ ]:
finaldf.head()

,id,age,gender,size,name,description,breeds.primary,breeds.secondary,breeds.mixed,breeds.unknown,primary_photo_cropped.full,contact.address.postcode
0,52120719,Adult,Male,Medium,Whiskey,"IF NOT CLAIMED BY OWNER AVAILABLE WEDNESDAY, J...",Dogue de Bordeaux,None,True,False,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,45761
1,51863872,Adult,Male,Large,Charlie,This pet&amp;#39;s expected availability date ...,Labrador Retriever,Chow Chow,True,False,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,60053
2,51863824,Adult,Female,Medium,Gabby,Gabby is a sweet 5-year-old gray pittie mix wh...,Pit Bull Terrier,None,True,False,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,60555
3,51863559,Adult,Male,Large,Gomez,"6 year old Neutered Male, lived with a single ...",Labrador Retriever,Rottweiler,True,False,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,48641
4,51863032,Adult,Male,Small,"PePe-St. Louis, MO!",PePe is a 9-10 year old POM. He is a very swee...,Pomeranian,None,False,False,https://dl5zpyw5k3jeb.cloudfront.net/photos/pe...,63132


In [ ]:
finaldf.to_csv('petfindernew.csv')

In [ ]:
import spacy
from spacy.matcher import Matcher

sp_lg = spacy.load('en_core_web_sm')

def getting_age_from_desc(desc):
  label_list = []
  for ent in sp_lg(desc).ents:
    label_list.append(ent.label_)
  if 'DATE' in label_list:
    for ent in sp_lg(desc).ents:
      if ent.label_ == 'DATE':
        return ent.text.strip()
  elif "CARDINAL" in label_list:
    for ent in sp_lg(desc).ents:
      if ent.label_ == 'CARDINAL':
        return ent.text.strip()
    

print(getting_age_from_desc("8 months "))

8 months


In [ ]:
string = '75'
print(string.isnumeric())

True


In [ ]:
import regex as re
possible_weight_words = ["pounds", "pound", "lbs", "lb", "oz"]
numdict = {'one':1,
             'two':2,
             'three':3,
             'four':4,
             'five':5,
             'six':6,
             'seven':7,
             'eight':8,
             'nine':9,
             'ten':10,
             'eleven':11}
             
def removechar(rawdesc,index):
  if rawdesc[index].isdigit():
    rawdesc = rawdesc[0:index:]+rawdesc[index+1::]
    return rawdesc
  else:
    return rawdesc


def removeNearbyNumbers(description,startindex):
  
  rawtext = description

  for indexint in startindex:
    if indexint != -1:
      for i in range(0,6):
        numindex = indexint-i
        rawtext = removechar(rawtext,numindex)

      for substring in possible_weight_words:
        if substring in rawtext:
          rawtext = rawtext.replace(substring, "")

  return rawtext

def getAge(description):
  age = re.findall(r'Age[\:\s](\d{1,2})', description)
  age.extend(re.findall(r' (\d{1,2}),? ', description))
  if len(age) == 0:
    age = re.findall(r'\((\d{1,2})\)', description)
  if len(age) == 0:
    age = re.findall(r'(\d+).*?(?i)(yrs)', description)
  if len(age) == 0:
    age = re.findall(r'(\d+)(?i)yr', description)
  if len(age) == 0:
    age = re.findall(r'[0-9]+-(year).*', description)
  #print(description+ " --- AGE: "+ str(set(age)))
  return getting_age_from_desc(description)

def getNumfromString(string):
  print(string.split())
  p = re.compile('^[0-9]*\.[0-9]+$')
  if p.match('8.5'):
    print('yes')
  age = [float(word) for word in string.split() if word.isdigit() or p.match(word)]
  print(age)
  if '1/2' in string:
    age.append(float(.5))
  if len(age) == 0:
    age = [float(numdict[numword]) for numword in string.split() if numword in [*numdict]]
  return age

def getAgefromString(string):
  print(string)
  numdict = {'one':1,
             'two':2,
             'three':3,
             'four':4,
             'five':5,
             'six':6,
             'seven':7,
             'eight':8,
             'nine':9,
             'ten':10,
             'eleven':11}
  possible_week = ["wk","week"]
  possible_months = ["month","mnth","mth"]
  possible_yrs= ["yr","years"]

  string = str(string)
  if string != "None":
    if any(map(string.__contains__, possible_yrs)):

      age = getNumfromString(string)
      return age

    if any(map(string.__contains__, possible_week)):

      age = [num/52 for num in getNumfromString(string)]
      return age 
    
    if any(map(string.__contains__, possible_months)):

      age = [num/12 for num in getNumfromString(string)] 
      return age
    else:
      age = getNumfromString(string)
      return age
  else:
    return None


In [ ]:
print(getAgefromString(getAge('8.5 weeks')))

8.5 weeks
['8.5', 'weeks']
yes
[8.5]
[0.16346153846153846]


In [ ]:
import re

possible_words = ["age","old", "yr", "aged", "year", "year-old", "month", "DOB", 'week']
possible_weight_words = ["pound", "lb", "oz"]
count = 0

finaldf['age_num'] = ''
for i in range(len(finaldf['description'])):
  if not pd.isnull(finaldf['description'][i]) :
    if any(map(finaldf['description'][i].__contains__, possible_words)):
      descriptionstring = finaldf['description'][i]
      if any(map(finaldf['description'][i].__contains__, possible_weight_words)):
        startendindex = []

        for substring in possible_weight_words:
          indexofsub = descriptionstring.find(substring)
          startendindex.append(int(indexofsub))

        descriptionstring = removeNearbyNumbers(descriptionstring,startendindex)
      finaldf.at[i,'age_num']= getAgefromString(getAge(descriptionstring))
    else:
      finaldf.at[i,'age_num']=None
  else:
    finaldf.at[i,'age_num']=None


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: Flags not at the start of the expression '(\\d+).*?(?i)(yrs)'
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: Flags not at the start of the expression '(\\d+)(?i)yr'


In [ ]:
finaldf.at[0,'name'] = 'Whiskey'

In [ ]:
finaldf.loc[84922,'name']

'Chad'

In [ ]:
def avg(lst):
  print(type(lst))
  return sum(lst)/len(lst)

for i in range(len(finaldf['age_num'])):
  if type(finaldf['age_num'][i]) == float or finaldf['age_num'][i] == None:
    continue
  elif len(finaldf['age_num'][i]) == 0:
    finaldf.at[i,'age_num'] = None
  elif len(finaldf['age_num'][i]) == 1:
    listhere = finaldf.loc[int(i),'age_num']
    finaldf.at[str(i),'age_num'] = int(listhere[0])
  else:
    finaldf.at[i,'age_num'] = avg(finaldf.loc[int(i),'age_num'])

In [ ]:
print(finaldf['age_num'].value_counts())

0                      3738
2.0                    1036
[2]                    1032
3.0                     740
[3]                     736
                       ... 
84.45833333333334         1
0.11057692307692307       1
0.10576923076923078       1
44                        1
5.75                      1
Name: age_num, Length: 143, dtype: int64


In [ ]:
finaldf.to_csv('petfindernew.csv')

In [ ]:
count = 0
for i in range(len(finaldf['age_num'])):
  if finaldf['age_num'][i] != None:
    count +=1

print(count)
print(len(finaldf['age_num']))

10195
84923


In [ ]:

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match IDs
pattern = [[{"TEXT": "Age"}, {"LIKE_NUM": True}],
           [{"TEXT": "Age"},{"IS_PUNCT": True}, {"LIKE_NUM": True}],
           [{"LIKE_NUM": True},{"TEXT": "years old "}]]

pattenrr =[
    [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}],
    [{"LOWER": "hello"}, {"LOWER": "world"}]
]
matcher.add("HelloWorld", pattern)


doc = nlp("Age 9")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

15578876784678163569 HelloWorld 0 2 Age 9
